# **'Precio Media Anual'** Propiedades

In [24]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "puebla" in nombre
    and ('financiamientos' not in nombre and 'poblacion' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre )
    and 'q' not in nombre
    
]  

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [25]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
        # Renombrar la columna
    df.rename(columns={'Recamaras':'recamaras','Estacionamiento':'estacionamiento','m_total': 'metros_total','m_construido':'metros_construido','tiempo_publicacion':'tiempo_de_publicacion','Estacionamientos':'estacionamiento','Recamaras': 'recamaras', 
       'baño':'Banos','medio_baño':'Medio_banos', 'Seguridad':'seguridad','seguridad_privada':'seguridad','baño_total':'Banos_Total', 'cp':'CP',                  'Colonia':'colonia','Estado':'estado','Tipo':'tipo','Categoria':'categoria','Precio':'precio','m2_contruido': 'm2_construido','segmento':'categoria','Categoria':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [26]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)

In [27]:
import re

# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

df_resultados = pd.DataFrame(columns=['categoria', 'precio', 'mes', 'año'])
for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Añadir las columnas de mes y año al DataFrame actual
        df['mes'] = mes
        df['año'] = ano
        df_resultados = pd.concat([df_resultados, df], ignore_index=True)
df_resultados[['categoria','precio','mes','año']]


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16260\1530539089.py:35: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,categoria,precio,mes,año
0,E1,280000.0,8,2024
1,E1,280000.0,8,2024
2,E1,315000.0,8,2024
3,E1,323656.0,8,2024
4,E1,324900.0,8,2024
...,...,...,...,...
32335,S,21000000.0,9,2023
32336,S,33000000.0,9,2023
32337,S,33000000.0,9,2023
32338,S,37000000.0,9,2023


# Regresión mensual (RF)

In [6]:
df_resultados = df_resultados.groupby(['mes','año'])['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,mes,año,montos
0,2,2024,2350000.0
1,3,2024,2550000.0
2,5,2024,2500000.0
3,6,2024,2480000.0
4,7,2023,3100000.0
5,7,2024,2317861.5
6,8,2024,2250000.0
7,9,2023,3000000.0
8,9,2024,2911856.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)

RandomForestRegressor()

In [8]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 3),
                            'mes': range(10, 13)})
fechas_20232024 = pd.DataFrame({'año': np.repeat(2024, 1),
                            'mes': range(1, 2)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 3),
                            'mes': range(10, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023,fechas_20232024,fechas_2024, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby(['año', 'mes'])['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,mes,montos
0,2023,7,3100000.00
1,2023,9,3000000.00
2,2023,10,2989701.12
3,2023,11,2989701.12
4,2023,12,2989701.12
5,2024,1,2389757.23
6,2024,2,2350000.00
7,2024,3,2550000.00
8,2024,5,2500000.00
9,2024,6,2480000.00


In [9]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
total_todos_pachuca['label'] = total_todos_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + total_todos_pachuca['año'].astype(str)
total_todos_pachuca = total_todos_pachuca.sort_values(by=['año', 'mes'])
total_todos_pachuca

,año,mes,montos,label
0,2023,7,3100000.00,julio-2023
1,2023,9,3000000.00,septiembre-2023
2,2023,10,2989701.12,octubre-2023
3,2023,11,2989701.12,noviembre-2023
4,2023,12,2989701.12,diciembre-2023
5,2024,1,2389757.23,enero-2024
6,2024,2,2350000.00,febrero-2024
7,2024,3,2550000.00,marzo-2024
8,2024,5,2500000.00,mayo-2024
9,2024,6,2480000.00,junio-2024


In [10]:
# Datos
año = total_todos_pachuca['label']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[900000, 5500000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[1900000, 2900000, 3900000, 5000000],  # Definir los valores de las marcas en el eje y un poco antes del millón
        ticktext=['$1,900,000.00','$2,900,000.00', '$3,900,000.00','$5,500,000.00'],  # Definir el texto de las marcas en el eje y un poco antes del millón
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

# Regresión mensual (RF)
## Cortando 2023

In [13]:
df_resultados = df_resultados[~df_resultados['año'].isin([2022, 2023])]
df_resultados = df_resultados.groupby(['mes','año'])['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,mes,año,montos
0,2,2024,2350000.0
1,3,2024,2550000.0
2,5,2024,2500000.0
3,6,2024,2480000.0
4,7,2024,2317861.5
5,8,2024,2250000.0
6,9,2024,2911856.0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)

RandomForestRegressor()

In [15]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_20232024 = pd.DataFrame({'año': np.repeat(2024, 1),
                            'mes': range(1, 2)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 3),
                            'mes': range(10, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_20232024,fechas_2024, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby(['año', 'mes'])['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,mes,montos
0,2024,1,2482371.69
1,2024,2,2350000.00
2,2024,3,2550000.00
3,2024,5,2500000.00
4,2024,6,2480000.00
5,2024,7,2317861.50
6,2024,8,2250000.00
7,2024,9,2911856.00
8,2024,10,2641341.25
9,2024,11,2641341.25


In [16]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
total_todos_pachuca['label'] = total_todos_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + total_todos_pachuca['año'].astype(str)
total_todos_pachuca = total_todos_pachuca.sort_values(by=['año', 'mes'])
total_todos_pachuca

,año,mes,montos,label
0,2024,1,2482371.69,enero-2024
1,2024,2,2350000.00,febrero-2024
2,2024,3,2550000.00,marzo-2024
3,2024,5,2500000.00,mayo-2024
4,2024,6,2480000.00,junio-2024
5,2024,7,2317861.50,julio-2024
6,2024,8,2250000.00,agosto-2024
7,2024,9,2911856.00,septiembre-2024
8,2024,10,2641341.25,octubre-2024
9,2024,11,2641341.25,noviembre-2024


In [17]:
# Datos
año = total_todos_pachuca['label']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[2000000, 3000000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[2000000, 2500000, 3000000],  # Añadir la mitad entre 2000000 y 3000000
        ticktext=['$2,000,000.00','$2,500,000.00','$3,000,000.00'],  # Añadir el texto para la marca de la mitad
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

# Regresión Anual

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['precio']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [29]:
df_resultados = df_resultados.groupby('año')['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,año,montos
0,2023,3050962.5
1,2024,2490000.0


In [30]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 3),
                            'mes': range(10, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,montos
0,2023,3050962.50
1,2024,4142120.72
2,2025,3654137.47
3,2026,3416100.95


In [31]:
# Datos
año = total_todos_pachuca['año']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[900000, 5500000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[1900000, 2900000, 3900000, 5000000],  # Definir los valores de las marcas en el eje y un poco antes del millón
        ticktext=['$1,900,000.00','$2,900,000.00', '$3,900,000.00','$5,500,000.00'],  # Definir el texto de las marcas en el eje y un poco antes del millón
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()